In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering

In [ ]:
# Importing dataset
data = pd.read_excel('https://github.com/rushikeshw791/Clustering-1/files/9793741/EastWestAirlines.xlsx',sheet_name='data')
data.head()

In [ ]:
# Column rename.
data.rename(columns={'ID#':'ID', 'Award?':'Award'}, inplace=True)
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data1=data.drop(labels='ID',axis=1)
data1.head()

In [ ]:
# correlation heatmap

f,ax = plt.subplots(figsize=(18,12))
sns.heatmap(data.corr(), annot=True, linewidths =.5, fmt ='.1f',ax=ax)
plt.show()

**Data preprocessing**

1.Standardizing the data

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [ ]:
standard_scaler = StandardScaler()
std_df = standard_scaler.fit_transform(data1)
std_df.shape

In [ ]:
plt.figure(figsize=(20,12))
dendo = sch.dendrogram(sch.linkage(std_df,method='single'))

In [ ]:
plt.figure(figsize=(20,12))
dendogram = sch.dendrogram(sch.linkage(std_df,method='complete'))

In [ ]:
plt.figure(figsize=(20,12))
dendogram = sch.dendrogram(sch.linkage(std_df,method='average'))

In [ ]:
plt.figure(figsize=(20,12))
dendogram = sch.dendrogram(sch.linkage(std_df,method='ward'))

2.Normalizing the data

In [ ]:
# Using Minmaxscaler for accuracy result comparison

minmax = MinMaxScaler()

minmax_df = minmax.fit_transform(data1)
minmax_df.shape

(3999, 11)

In [ ]:
plt.figure(figsize=(20,12))
dendogram = sch.dendrogram(sch.linkage(minmax_df,method='single'))

In [ ]:
plt.figure(figsize=(20,12))
dendogram = sch.dendrogram(sch.linkage(minmax_df,method='complete'))

In [ ]:
plt.figure(figsize=(20,12))
dendogram = sch.dendrogram(sch.linkage(minmax_df,method='average'))

In [ ]:
plt.figure(figsize=(20,12))
dendogram = sch.dendrogram(sch.linkage(minmax_df,method='ward'))

**Create clusters (y)**

In [ ]:
hclusters=AgglomerativeClustering(n_clusters=5,affinity='euclidean',linkage='ward')
hclusters

In [ ]:
y_hc=hclusters.fit_predict(minmax_df)

In [ ]:
y_hc

array([1, 1, 1, ..., 0, 4, 4])

In [ ]:
data['h_clusterid']=hclusters.labels_

In [ ]:
data

In [ ]:
y=pd.DataFrame(hclusters.fit_predict(minmax_df),columns=['clusterid'])
y['clusterid'].value_counts()

In [ ]:
# adding clusters to the dataset
data1['clusterid']=hclusters.labels_
data1

In [ ]:
data1.groupby('clusterid').agg(['mean']).reset_index()

In [ ]:
# plot clusters
plt.figure(figsize=(10,7))
plt.scatter(data1['clusterid'],data1['Balance'],c=hclusters.labels_)

**K means**

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [ ]:
airline1= pd.read_excel('https://github.com/rushikeshw791/Clustering-1/files/9793741/EastWestAirlines.xlsx',sheet_name='data')
airline1.head()

In [ ]:
#Normalized data fuction
def norm_func(i):
    x=(i-i.min())/(i.max()-i.min())
    return(x)

In [ ]:
df_norm = norm_func(airline1.iloc[:,1:])

In [ ]:
# Elbow curve
wcss=[]
for i in range(1,5):
  kmeans=KMeans(n_clusters=i)
  kmeans.fit(df_norm)
  wcss.append(kmeans.inertia_)
plt.plot(range(1,5),wcss)
plt.title('Elbow Curve')
plt.xlabel('Number of clusters')
plt.ylabel('wcss')
plt.show()

In [ ]:
# select 4 clusters from above plot
model=KMeans(n_clusters=4)
model.fit(df_norm)
model.labels_

array([0, 0, 0, ..., 3, 0, 0], dtype=int32)

In [ ]:
x=pd.Series(model.labels_)
airline1['Clust']=x

In [ ]:
airline1

**DBSCAN**

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
airline2 = pd.read_excel('https://github.com/rushikeshw791/Clustering-1/files/9793741/EastWestAirlines.xlsx',sheet_name='data')
airline2.head()

In [ ]:
airline2.info()

In [ ]:
df=airline2.iloc[:,1:5]

In [ ]:
df.values

In [ ]:
stscaler=StandardScaler().fit(df.values)
x=stscaler.transform(df.values)

In [ ]:
x

In [ ]:
dbscan=DBSCAN(eps=2,min_samples=5)
dbscan.fit(x)

In [ ]:
dbscan.labels_

In [ ]:
c1=pd.DataFrame(dbscan.labels_,columns=['cluster'])

In [ ]:
c1

In [ ]:
pd.concat([airline2,c1],axis=1)